## Clip and merge tif tiles

Raster images are usually very large. Therefore, we usually cut a large image into smaller tiles. If we want get the big picture, we usually merge smaller tiles into big one. This notebook includes two sections, clip and merge. 

### 1. Clip big image into smaller tiles

In [17]:
import rasterio
import os, os.path
from rasterio.windows import Window
from rasterio.merge import merge


In [7]:
lu_dataset = rasterio.open('cir-naip/pa_m_4007562_ne_18_1_20170910_20171207.tif')
outfolder = 'naip-tiles'
if not os.path.exists(outfolder):
    os.mkdir(outfolder)

lu_img = lu_dataset.read()

# height and width of the raster
height, width = lu_dataset.shape

# split the large raser into number of tiles, tile size is 2000*2000
tile_width = 600
tile_height = 600

tile_num_col = int(width/tile_width) + 1
tile_num_row = int(height/tile_height) + 1

# the output folder
if not os.path.exists(outfolder): os.mkdir(outfolder)

# create each tile
for i in range(tile_num_row - 1):
    # loop each column
    for j in range(tile_num_col - 1):
        print('The i and j is:', i, j)
        tilename = os.path.join(outfolder, 'row%s-col%s.tif'%(i, j))
        
        row_start = i*tile_height
        col_start = j*tile_width
        row_end = (i + 1)*tile_height
        col_end = (j + 1)*tile_width
        
        # deal with the right boundary
        if row_end > height - 1: tile_height = height - row_start
        if col_end > width - 1: tile_width = width - col_start
        
        # window = get_data_window(lu_dataset.read(1, masked=True))
        window = Window(col_off=col_start, row_off=row_start, width=tile_width, height=tile_height)
        
        kwargs = lu_dataset.meta.copy()
        kwargs.update({
            'height': window.height,
            'width': window.width,
            'transform': rasterio.windows.transform(window, lu_dataset.transform)})
        
        with rasterio.open(tilename, 'w', **kwargs) as dst:
            dst.write(lu_dataset.read(window=window))
        

The i and j is: 0 0
The i and j is: 0 1
The i and j is: 0 2
The i and j is: 0 3
The i and j is: 0 4
The i and j is: 0 5
The i and j is: 0 6
The i and j is: 0 7
The i and j is: 0 8
The i and j is: 1 0
The i and j is: 1 1
The i and j is: 1 2
The i and j is: 1 3
The i and j is: 1 4
The i and j is: 1 5
The i and j is: 1 6
The i and j is: 1 7
The i and j is: 1 8
The i and j is: 2 0
The i and j is: 2 1
The i and j is: 2 2
The i and j is: 2 3
The i and j is: 2 4
The i and j is: 2 5
The i and j is: 2 6
The i and j is: 2 7
The i and j is: 2 8
The i and j is: 3 0
The i and j is: 3 1
The i and j is: 3 2
The i and j is: 3 3
The i and j is: 3 4
The i and j is: 3 5
The i and j is: 3 6
The i and j is: 3 7
The i and j is: 3 8
The i and j is: 4 0
The i and j is: 4 1
The i and j is: 4 2
The i and j is: 4 3
The i and j is: 4 4
The i and j is: 4 5
The i and j is: 4 6
The i and j is: 4 7
The i and j is: 4 8
The i and j is: 5 0
The i and j is: 5 1
The i and j is: 5 2
The i and j is: 5 3
The i and j is: 5 4


### 2. Merge tiles into big image

In [11]:
ls naip-tiles

row0-col0.tif   row10-col4.tif  row2-col8.tif   row5-col3.tif   row7-col7.tif
row0-col1.tif   row10-col5.tif  row3-col0.tif   row5-col4.tif   row7-col8.tif
row0-col2.tif   row10-col6.tif  row3-col1.tif   row5-col5.tif   row8-col0.tif
row0-col3.tif   row10-col7.tif  row3-col2.tif   row5-col6.tif   row8-col1.tif
row0-col4.tif   row10-col8.tif  row3-col3.tif   row5-col7.tif   row8-col2.tif
row0-col5.tif   row11-col0.tif  row3-col4.tif   row5-col8.tif   row8-col3.tif
row0-col6.tif   row11-col1.tif  row3-col5.tif   row6-col0.tif   row8-col4.tif
row0-col7.tif   row11-col2.tif  row3-col6.tif   row6-col1.tif   row8-col5.tif
row0-col8.tif   row11-col3.tif  row3-col7.tif   row6-col2.tif   row8-col6.tif
row1-col0.tif   row11-col4.tif  row3-col8.tif   row6-col3.tif   row8-col7.tif
row1-col1.tif   row11-col5.tif  row4-col0.tif   row6-col4.tif   row8-col8.tif
row1-col2.tif   row11-col6.tif  row4-col1.tif   row6-col5.tif   row9-col0.tif
row1-col3.tif   row11-col7.tif  row4-col2.tif   row6-col6.tif   

In [19]:
folder = 'naip-tiles'

src_files_to_mosaic = []

for file in os.listdir(folder):
    if not file.endswith('.tif'): continue
    filename = os.path.join(folder, file)
    
    tile_src = rasterio.open(filename)
    src_files_to_mosaic.append(tile_src)

print('The number of mosaiced tiles is-------------:', len(src_files_to_mosaic))

The number of mosaiced tiles is-------------: 108


In [20]:
# the method can be set as min, max..
mosaic, out_trans = merge(src_files_to_mosaic) #, method='max'
print('You have mosaiced the results')
print(mosaic.shape[1], mosaic.shape[2])

# Prepare the schema of the output mosacied image
out_meta = tile_src.meta.copy()
out_meta.update({"driver": "GTiff",
                  "height": mosaic.shape[1],
                  "width": mosaic.shape[2],
                  "transform": out_trans, 
                  "compress": 'lzw',
                  'BIGTIFF': 'YES'
                  }
               )

outfile = 'mosaiced.tif'

#out_fp = os.path.join(dirpath, 'atlanta-naip.tif')
with rasterio.open(outfile, "w", **out_meta) as dest:
     dest.write(mosaic)
    

You have mosaiced the results
7200 5400
